# Qwen3-TTS — 無料ボイスクローン & 音声生成

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ysd416jp/-yy_crawler/blob/claude/mobile-tts-system-JEkXZ/Qwen3_TTS_Colab.ipynb)

Alibaba Qwen3-TTS を Google Colab の無料 GPU (T4) で動かします。

**主な機能:**
- **ボイスクローン**: mp3/wavをアップ → 日本語テキストを打つ → その声で喋る
- **TTS**: プリセット音声でテキスト読み上げ
- **ボイスデザイン**: 自然言語で声質を指定して生成

## スマホからの使い方（3ステップ）

1. 「ランタイム」→「ランタイムのタイプを変更」→ **T4 GPU** を選択
2. 「ランタイム」→「**すべてのセルを実行**」をタップ
3. 最後のセルに表示される `https://xxxxx.gradio.live` のURLをタップ → 完了!

## 1. GPU確認 & インストール

「ランタイム」→「ランタイムのタイプを変更」→ **T4 GPU** を選択してください。

In [ ]:
# GPU確認
!nvidia-smi
print("\n" + "="*60)

import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu_name} ({gpu_mem:.1f} GB)")
    print("OK: GPUが利用可能です")
else:
    print("WARNING: GPUが見つかりません。")
    print("「ランタイム」→「ランタイムのタイプを変更」→ T4 GPU を選択してください")

In [ ]:
# Qwen3-TTS と関連パッケージをインストール
!pip install -U qwen-tts gradio soundfile numpy -q

print("\nインストール完了")

## 2. モデル設定

**モデルサイズを選択:**
- `0.6B`: 軽量・高速。無料T4で余裕で動く
- `1.7B`: 高音質。無料T4 (16GB) でも動作可能

In [ ]:
#@title モデルサイズ選択 { run: "auto" }
MODEL_SIZE = "1.7B"  #@param ["0.6B", "1.7B"]

print(f"選択: {MODEL_SIZE} モデル")
if MODEL_SIZE == "1.7B":
    print("高音質モード。ダウンロードに数分かかります。")
else:
    print("軽量モード。高速に起動します。")

## 3. モデル読み込み

初回実行時はモデルのダウンロードが入ります（数分）。

In [ ]:
import torch
from qwen_tts import Qwen3TTSModel
import soundfile as sf
import numpy as np
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16

# FlashAttention 2: Ampere (compute capability 8.0) 以降のGPUのみ対応
# T4 は Turing (7.5) なので使えない → SDPA を使用
attn_impl = "sdpa"
if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8:
    try:
        import flash_attn
        attn_impl = "flash_attention_2"
        print("FlashAttention 2: 有効 (Ampere GPU)")
    except ImportError:
        print("FlashAttention 2: 未インストール → SDPA使用")
else:
    print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
    print(f"FlashAttention 2: 非対応GPU → SDPA使用（問題なし）")

print(f"Device: {device}, Dtype: {dtype}, Attention: {attn_impl}")
print(f"\n--- CustomVoice モデル読み込み中... ---")

# CustomVoice: プリセット音声でTTS
model_custom = Qwen3TTSModel.from_pretrained(
    f"Qwen/Qwen3-TTS-12Hz-{MODEL_SIZE}-CustomVoice",
    device_map=device,
    dtype=dtype,
    attn_implementation=attn_impl,
)
print("CustomVoice OK")

print(f"\n--- Base モデル読み込み中... ---")

# Base: ボイスクローン
model_base = Qwen3TTSModel.from_pretrained(
    f"Qwen/Qwen3-TTS-12Hz-{MODEL_SIZE}-Base",
    device_map=device,
    dtype=dtype,
    attn_implementation=attn_impl,
)
print("Base OK")

# VoiceDesign: 1.7Bのみ
model_design = None
if MODEL_SIZE == "1.7B":
    print(f"\n--- VoiceDesign モデル読み込み中... ---")
    model_design = Qwen3TTSModel.from_pretrained(
        "Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign",
        device_map=device,
        dtype=dtype,
        attn_implementation=attn_impl,
    )
    print("VoiceDesign OK")

# VRAM確認
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"\nVRAM: {allocated:.1f} / {total:.1f} GB")

print("\n全モデル読み込み完了")

## 4. Gradio UI 起動

実行すると公開 URL が表示されます。  
その URL をスマホで開けば、どこからでも使えます。

In [ ]:
import gc
import json
import random
import re
import gradio as gr
import soundfile as sf
import numpy as np
import tempfile
import os

# ==================================================
# モデル状態リセット（2回目以降の生成で音声が破綻する問題の修正）
# ==================================================
def _reset_model_state(model):
    """生成間で残留する内部状態をクリアし、2回目以降の音声破綻を防ぐ。"""
    inner = getattr(model, "model", None)
    if inner is None:
        return
    if hasattr(inner, "rope_deltas"):
        inner.rope_deltas = None
    if hasattr(inner, "generation_step"):
        inner.generation_step = -1
    if hasattr(inner, "past_hidden"):
        inner.past_hidden = None
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()


def _set_seed(seed):
    """乱数シードを固定して再現性のある生成を行う。"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def _sampling_kwargs(temperature, top_p):
    """サンプリングパラメータの dict を返す。"""
    return dict(
        temperature=temperature,
        top_p=top_p,
        top_k=50,
        repetition_penalty=1.0,
        subtalker_temperature=temperature,
        subtalker_top_p=top_p,
        subtalker_top_k=50,
    )


# ==================================================
# 読み替え辞書
# ==================================================
DICT_PATH = "tts_dictionary.json"

def _load_dictionary():
    if os.path.exists(DICT_PATH):
        with open(DICT_PATH, encoding="utf-8") as f:
            return json.load(f)
    return {}

def _save_dictionary(dic):
    with open(DICT_PATH, "w", encoding="utf-8") as f:
        json.dump(dic, f, ensure_ascii=False, indent=2)

def _apply_dictionary(text):
    dic = _load_dictionary()
    if not dic:
        return text
    pattern = re.compile("|".join(re.escape(k) for k in sorted(dic, key=len, reverse=True)))
    return pattern.sub(lambda m: dic[m.group()], text)


# プリセット音声リスト
SPEAKERS = ["Vivian", "Serena", "Uncle_Fu", "Dylan", "Eric", "Ryan", "Aiden", "Ono_Anna", "Sohee"]

LANGUAGES = {
    "日本語": "Japanese",
    "英語": "English",
    "中国語": "Chinese",
    "韓国語": "Korean",
    "ドイツ語": "German",
    "フランス語": "French",
}

# ボイスクローン用プロンプトキャッシュ
_clone_prompt_cache = {"ref_audio": None, "ref_text": None, "prompt": None}


def voice_clone(text, language, ref_audio, ref_text, temperature, top_p, seed):
    """参照音声からクローン生成"""
    if not text.strip():
        raise gr.Error("読み上げテキストを入力してください")
    if ref_audio is None:
        raise gr.Error("参照音声(mp3/wav)をアップロードしてください")
    if not ref_text.strip():
        raise gr.Error("参照音声のテキスト（何と言っているか）を入力してください")
    try:
        _reset_model_state(model_base)
        if seed >= 0:
            _set_seed(int(seed))
        text = _apply_dictionary(text)
        lang = LANGUAGES.get(language, "Japanese")
        kwargs = _sampling_kwargs(temperature, top_p)
        # 同じ参照音声ならプロンプトを再利用（声質の安定性向上）
        if (_clone_prompt_cache["ref_audio"] != ref_audio
                or _clone_prompt_cache["ref_text"] != ref_text):
            _clone_prompt_cache["prompt"] = model_base.create_voice_clone_prompt(
                ref_audio=ref_audio, ref_text=ref_text,
            )
            _clone_prompt_cache["ref_audio"] = ref_audio
            _clone_prompt_cache["ref_text"] = ref_text
        wavs, sr = model_base.generate_voice_clone(
            text=text,
            language=lang,
            voice_clone_prompt=_clone_prompt_cache["prompt"],
            **kwargs,
        )
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        sf.write(tmp.name, wavs[0], sr)
        return tmp.name
    except Exception as e:
        raise gr.Error(f"エラー: {e}")


def tts_generate(text, speaker, language, temperature, top_p, seed):
    """プリセット音声で読み上げ"""
    if not text.strip():
        raise gr.Error("テキストを入力してください")
    try:
        _reset_model_state(model_custom)
        if seed >= 0:
            _set_seed(int(seed))
        text = _apply_dictionary(text)
        lang = LANGUAGES.get(language, "Japanese")
        kwargs = _sampling_kwargs(temperature, top_p)
        wavs, sr = model_custom.generate_custom_voice(
            text=text,
            language=lang,
            speaker=speaker,
            **kwargs,
        )
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        sf.write(tmp.name, wavs[0], sr)
        return tmp.name
    except Exception as e:
        raise gr.Error(f"エラー: {e}")


def voice_design(text, language, instruct_text, temperature, top_p, seed):
    """自然言語で声質を指定して生成"""
    if model_design is None:
        raise gr.Error("VoiceDesignは1.7Bモデルのみ対応です")
    if not text.strip():
        raise gr.Error("テキストを入力してください")
    if not instruct_text.strip():
        raise gr.Error("声質の説明を入力してください")
    try:
        _reset_model_state(model_design)
        if seed >= 0:
            _set_seed(int(seed))
        text = _apply_dictionary(text)
        lang = LANGUAGES.get(language, "Japanese")
        kwargs = _sampling_kwargs(temperature, top_p)
        wavs, sr = model_design.generate_voice_design(
            text=text,
            language=lang,
            instruct=instruct_text,
            **kwargs,
        )
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        sf.write(tmp.name, wavs[0], sr)
        return tmp.name
    except Exception as e:
        raise gr.Error(f"エラー: {e}")


# --- 辞書管理用ヘルパー ---
def _dict_to_table():
    dic = _load_dictionary()
    if not dic:
        return []
    return [[k, v] for k, v in dic.items()]

def _add_entry(word, reading):
    if not word.strip() or not reading.strip():
        return _dict_to_table(), "元の表記と読み替えの両方を入力してください"
    dic = _load_dictionary()
    dic[word.strip()] = reading.strip()
    _save_dictionary(dic)
    return _dict_to_table(), f"登録: {word.strip()} → {reading.strip()}"

def _del_entry(word):
    if not word.strip():
        return _dict_to_table(), "削除する元の表記を入力してください"
    dic = _load_dictionary()
    if word.strip() in dic:
        del dic[word.strip()]
        _save_dictionary(dic)
        return _dict_to_table(), f"削除: {word.strip()}"
    return _dict_to_table(), f"「{word.strip()}」は辞書に未登録です"


# --- Gradio UI (スマホ対応) ---
with gr.Blocks(
    title="Qwen3-TTS ボイスクローン",
    theme=gr.themes.Soft(primary_hue="blue", neutral_hue="slate"),
) as demo:
    gr.Markdown("# Qwen3-TTS 音声生成")
    gr.Markdown(f"モデル: **{MODEL_SIZE}** | GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

    # === ボイスクローンタブ (最初に表示) ===
    with gr.Tab("ボイスクローン"):
        gr.Markdown("**mp3/wavをアップ → テキストを入力 → その声で喋る**")

        clone_ref_audio = gr.Audio(
            label="① 参照音声をアップロード (3秒以上のmp3/wav)",
            type="filepath",
        )
        clone_ref_text = gr.Textbox(
            label="② 参照音声のテキスト (音声で何と言っているか)",
            placeholder="例: こんにちは、今日はいい天気ですね。",
            lines=2,
        )

        gr.Markdown("---")

        clone_text = gr.Textbox(
            label="③ 読み上げたいテキスト (この声で喋らせたい内容)",
            placeholder="クローンした声で読み上げたいテキストを入力...",
            lines=4,
            value="こんにちは。この声は、アップロードされた音声からクローンされたものです。いかがでしょうか？",
        )
        clone_lang = gr.Dropdown(
            choices=list(LANGUAGES.keys()),
            value="日本語",
            label="言語",
        )
        with gr.Accordion("生成設定", open=False):
            clone_temp = gr.Slider(
                minimum=0.1, maximum=1.0, value=0.7, step=0.05,
                label="Temperature（低い=安定、高い=表現豊か）",
            )
            clone_top_p = gr.Slider(
                minimum=0.5, maximum=1.0, value=0.85, step=0.05,
                label="Top-P（低い=安定、1.0=制限なし）",
            )
            clone_seed = gr.Number(
                value=-1, precision=0,
                label="シード（-1=ランダム、0以上=固定で再現可能）",
            )
        clone_btn = gr.Button("クローン音声を生成", variant="primary", size="lg")
        clone_output = gr.Audio(label="生成結果", type="filepath")

        clone_btn.click(
            voice_clone,
            [clone_text, clone_lang, clone_ref_audio, clone_ref_text,
             clone_temp, clone_top_p, clone_seed],
            clone_output,
        )

    # === TTS読み上げタブ ===
    with gr.Tab("TTS 読み上げ"):
        gr.Markdown("プリセット音声でテキストを読み上げます")

        tts_text = gr.Textbox(
            label="テキスト",
            placeholder="読み上げたいテキストを入力...",
            lines=5,
            value="こんにちは。Qwen3-TTSのテストです。日本語の読み上げはいかがでしょうか？",
        )
        with gr.Row():
            tts_speaker = gr.Dropdown(
                choices=SPEAKERS,
                value="Ono_Anna",
                label="音声",
            )
            tts_lang = gr.Dropdown(
                choices=list(LANGUAGES.keys()),
                value="日本語",
                label="言語",
            )
        with gr.Accordion("生成設定", open=False):
            tts_temp = gr.Slider(
                minimum=0.1, maximum=1.0, value=0.7, step=0.05,
                label="Temperature（低い=安定、高い=表現豊か）",
            )
            tts_top_p = gr.Slider(
                minimum=0.5, maximum=1.0, value=0.85, step=0.05,
                label="Top-P（低い=安定、1.0=制限なし）",
            )
            tts_seed = gr.Number(
                value=-1, precision=0,
                label="シード（-1=ランダム、0以上=固定で再現可能）",
            )
        tts_btn = gr.Button("音声を生成", variant="primary", size="lg")
        tts_output = gr.Audio(label="生成結果", type="filepath")

        tts_btn.click(
            tts_generate,
            [tts_text, tts_speaker, tts_lang, tts_temp, tts_top_p, tts_seed],
            tts_output,
        )

    # === ボイスデザインタブ (1.7Bのみ) ===
    if model_design is not None:
        with gr.Tab("ボイスデザイン"):
            gr.Markdown("自然言語で声質を指定して音声を生成します")

            design_text = gr.Textbox(
                label="読み上げテキスト",
                placeholder="読み上げたいテキスト...",
                lines=4,
                value="今日はいい天気ですね。散歩に行きましょう。",
            )
            design_lang = gr.Dropdown(
                choices=list(LANGUAGES.keys()),
                value="日本語",
                label="言語",
            )
            design_desc = gr.Textbox(
                label="声質の説明 (英語推奨)",
                placeholder="例: A young Japanese woman with a gentle and warm voice",
                lines=3,
                value="A young Japanese woman with a gentle and warm voice, speaking clearly and naturally.",
            )
            with gr.Accordion("生成設定", open=False):
                design_temp = gr.Slider(
                    minimum=0.1, maximum=1.0, value=0.7, step=0.05,
                    label="Temperature（低い=安定、高い=表現豊か）",
                )
                design_top_p = gr.Slider(
                    minimum=0.5, maximum=1.0, value=0.85, step=0.05,
                    label="Top-P（低い=安定、1.0=制限なし）",
                )
                design_seed = gr.Number(
                    value=-1, precision=0,
                    label="シード（-1=ランダム、0以上=固定で再現可能）",
                )
            design_btn = gr.Button("デザイン生成", variant="primary", size="lg")
            design_output = gr.Audio(label="生成結果", type="filepath")

            design_btn.click(
                voice_design,
                [design_text, design_lang, design_desc,
                 design_temp, design_top_p, design_seed],
                design_output,
            )

    # === 読み替え辞書タブ ===
    with gr.Tab("読み替え辞書"):
        gr.Markdown(
            "イントネーションや読みがおかしい単語を登録すると、"
            "TTS 生成前に自動で置換されます。\n\n"
            "例: `AWS` → `エーダブリューエス` / `齟齬` → `そご`"
        )

        dict_table = gr.Dataframe(
            headers=["元の表記", "読み替え"],
            datatype=["str", "str"],
            value=_dict_to_table,
            label="登録済み辞書",
            interactive=False,
            row_count=(1, "dynamic"),
        )

        with gr.Row():
            dict_word = gr.Textbox(label="元の表記", placeholder="例: AWS")
            dict_reading = gr.Textbox(label="読み替え", placeholder="例: エーダブリューエス")
        with gr.Row():
            dict_add_btn = gr.Button("追加 / 更新", variant="primary")
            dict_del_btn = gr.Button("削除", variant="stop")

        dict_status = gr.Textbox(label="結果", interactive=False)

        dict_add_btn.click(_add_entry, [dict_word, dict_reading], [dict_table, dict_status])
        dict_del_btn.click(_del_entry, [dict_word], [dict_table, dict_status])

        gr.Markdown("---")
        gr.Markdown(
            "**ヒント**: 辞書は `tts_dictionary.json` に保存されます。"
            "テキストエディタで直接編集も可能です。"
        )

    gr.Markdown("---")
    gr.Markdown("Powered by [Qwen3-TTS](https://github.com/QwenLM/Qwen3-TTS) | 無料 Google Colab T4 GPU")

# 起動 (share=True で公開URLを発行 → スマホからアクセス可能)
print("\n" + "="*60)
print("Gradio UI を起動中...")
print("下に表示される https://xxxxx.gradio.live の URL をタップしてください")
print("="*60 + "\n")
demo.launch(share=True, quiet=False)

## 使い方

1. 上のセルを実行すると `https://xxxxx.gradio.live` のような URL が表示されます
2. その URL をスマホのブラウザで開く

### ボイスクローン（メイン機能）
1. **① 参照音声をアップロード**: スマホで録音したmp3/wavファイル（3秒以上）
2. **② 参照テキスト**: その音声で何と言っているかを入力
3. **③ 読み上げテキスト**: クローンした声で喋らせたい内容を入力
4. 「クローン音声を生成」ボタンを押す

### TTS読み上げ
- テキストを入力して音声を選ぶだけ（日本語なら **Ono_Anna** がおすすめ）

### ボイスデザイン (1.7Bのみ)
- 英語で声質を指定: 「A young woman with soft voice」のように記述

### プリセット音声の特徴
| 音声 | 特徴 |
|------|------|
| Ono_Anna | 女性・日本語向け |
| Vivian | 女性・英語ネイティブ風 |
| Serena | 女性・落ち着いたトーン |
| Dylan | 男性・英語ネイティブ風 |
| Eric | 男性・低音 |
| Ryan | 男性・ニュースキャスター風 |
| Aiden | 男性・若者風 |

### 注意事項
- Colabの無料枠ではGPU使用時間に制限があります
- 公開URLは72時間で失効します
- セッションが切れたら再実行してください